In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import PIL
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
from tqdm import tqdm
import itertools
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Add, Conv2D, MaxPooling2D,GlobalAveragePooling2D,ZeroPadding2D,Input, AveragePooling2D
from keras.models import Sequential
import sklearn.metrics as metrics

from numpy.random import seed
#from tensorflow import set_random_seed
seed_value=42
seed(42)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED']=str(seed_value)
import random 
random.seed(seed_value)
np.random.seed(seed_value)
print(seed_value)

In [ ]:
def load_data():
    
    class_names = ['Benign', 'Malignant_Melanoma', 'Malignant_Non_Melanoma']
    class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

    nb_classes = len(class_names)

    
    
    paths = [r'C:\Users\David\Desktop\Capstone_Dataset\training_valid', r'C:\Users\David\Desktop\Capstone_Dataset\testing']
    output = []
    
    
    for dataset in paths:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
       
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
          
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                img_path = os.path.join(os.path.join(dataset, folder), file)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (100,100)) 
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
def create_model():    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(100, 100, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2, seed=seed_value))
    #model.add(layers.Dropout(0.2,seed=seed_value))

    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2, seed=seed_value))
    
    #model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
   # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
   # model.add(MaxPooling2D((2, 2)))
    #model.add(Dropout(0.30))

    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2, seed=seed_value))
    model.add(Dense(3, activation='softmax'))

    opt = Adam(lr=0.001)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print (model.summary())
    
    return model


In [ ]:
def plot_confusion_matrix(cm, classes,
                         normalize=False,
                         title='Confusion Matrix',
                         cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float')/ cm.sum(axis=1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else: 
        print("Confusion matrix, without Normalization")
    print(cm)
    
    thresh = cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,cm[i,j],
                 horizontalalignment='center',
                 color="white" if cm[i,j] > thresh else "black")
        plt.tight_layout()
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')

In [ ]:
def train_model():

    (train_data, train_labels), (test_data, test_labels) = load_data()
    train_data, train_labels = shuffle(train_data, train_labels, random_state=seed_value)
    
    new_train = train_labels.shape[0]
    new_test = test_labels.shape[0]

    print ("Number of training examples: {}".format(new_train))
    print ("Number of testing examples: {}".format(new_test))
    print ("Each image is of size: {}".format((100,100)))

    train_data = train_data / 255.0 
    
    test_data = test_data / 255.0
    from sklearn.model_selection import train_test_split

    # Split the data
    train_data, valid_data, train_labels, valid_labels = train_test_split(train_data, train_labels, test_size=0.33, shuffle= True, random_state=seed_value)
    
    history = model.fit(train_data, train_labels, batch_size=64, epochs=20, validation_data=(valid_data, valid_labels))
    plt.subplot(121)
    plt.plot(history.history['accuracy'], label = "train accuracy")
    plt.plot(history.history['val_accuracy'], label = "val accuracy")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()
    plt.show()

    # Plot loss function
    plt.subplot(122)
    plt.plot(history.history['loss'], label = "train loss")
    plt.plot(history.history['val_loss'], label = "val loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()
    test_loss = model.evaluate(test_data, test_labels)
    #report2 = metrics.classification_report(true_classes, predicted_classes_x, target_names=class_labels)
    #print(report2)
    
    predictions = model.predict(x=test_data, verbose=0)
    cm=confusion_matrix(y_true=test_labels, y_pred=np.argmax(predictions, axis=-1))
    cm_plot_labels = ['Benign', 'M_M', 'M_N_M']
    plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
    print(classification_report(test_labels, np.argmax(predictions, axis=-1)))
    

In [ ]:
model = create_model()
train_model()